# CFT – Saliva Analysis

## Setup and Helper Functions

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp
from biopsykit.protocols import MIST

from cft_analysis.datasets import CftDatasetProcessed

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Import Data

In [ ]:
# get path to analysis results
base_path = Path("../../data")

In [ ]:
results_path = base_path.joinpath("../results")
stats_path = results_path.joinpath("statistics")
tex_path = stats_path.joinpath("tex_tables")
bp.utils.file_handling.mkdirs([results_path, stats_path, tex_path])

paper_path = Path("../paper_path.json")
paper_tex_path = None
if paper_path.exists():
    paper_path = Path(json.load(paper_path.open(encoding="utf-8"))["paper_path"])
    paper_tex_path = paper_path.joinpath("tab")
    bp.utils.file_handling.mkdirs([paper_tex_path])

In [ ]:
dataset = CftDatasetProcessed(base_path, exclude_subjects=True)
dataset

In [ ]:
cort_samples = dataset.cortisol
cort_features = dataset.cortisol_features

In [ ]:
sample_times = dataset.sample_times

hue_order = ["Control", "CFT"]

In [ ]:
mist = MIST.from_file(base_path.joinpath("mist_cft.json"))
mist

In [ ]:
mist.add_saliva_data(cort_samples, "cortisol", sample_times=sample_times)

Drop cortisol sample `S0` for further analysis (only used to check for high cortisol baseline)

In [ ]:
cort_samples = cort_samples.drop("S0", level="sample")

## Descriptive Analysis

### Cortisol Increase

Mean cortisol increase after MIST:

In [ ]:
cort_max_inc = cort_features.xs("max_inc_percent", level="saliva_feature")

cort_max_inc.groupby("condition").agg(["mean", "std"])

## Statistical Analysis

### MIST-induced Cortisol Increase

**Population**: Control group

**Analysis**: Check whether cortisol increased significantly after the MIST
* Procedure: Paired t-tests between cortisol sample before MIST (`S1`) and the maximum of all samples after MIST (`S2`-`S6`)
* Expected Result: Significant cortisol increase as response fo the MIST

**Findings**: 
* The MIST causes an effective activation of the HPA axis, indicated by a significant increase in cortisol after the MIST for the Control group

In [ ]:
display(Markdown("#### Prepare Data"))

# get S1
cort_s1 = cort_samples.xs("S1", level="sample")
# get maximum cortisol sample after MIST
cort_max = cort_samples.drop(["S0", "S1"], level="sample").groupby(["condition", "subject"]).max()
cort_analysis = pd.concat({"Pre": cort_s1, "Post": cort_max}, names=["time"])
cort_analysis = cort_analysis.reorder_levels(["condition", "subject", "time"]).sort_index()

display(cort_analysis.head())

################################################
display(Markdown("#### Statistics"))

steps = [("prep", "normality"), ("test", "pairwise_ttests")]
params = {"dv": "cortisol", "within": "time", "subject": "subject", "groupby": "condition"}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_response_mist.xlsx"))
stats.display_results()


################################################
display(Markdown("#### Latex Output"))

display(stats.stats_to_latex("pairwise_ttests"))

### Effect of CFT on Acute Stress Response

**Population**: Control vs. CFT condition

**Analysis**: 
1. *Interaction Condition x Time*: Check whether the CFT intervention interacts with HPA axis activity
    * Procedure:
        * Mixed-ANOVA to determine interaction effect between *Condition* and *Time*
        * In case of significant interaction effect: Post-hoc test to assess during which at sampling times cortisol samples were significantly different
    * Expected Result: Significant interaction effect between *Condition* and *Time*, cortisol levels start to significantly differ after the MIST

2. *Cortisol Features*: Check whether the CFT intervention causes an inhibition of the HPA axis, leading to less cortisol secretion
    * Procedure (for each cortisol feature): 
        * t-test between Control and CFT condition
    * Expected Result: Significant differences between Control and CFT condition: lower AUC, lower slope, lower maximum cortisol increase


**Findings**: 
* The MIST causes an effective activation of the HPA axis, indicated by lower AUC, lower slope and lower maximum cortisol response

#### Interaction *Condition* x *Time*

In [ ]:
display(Markdown("##### Prepare Data"))

cort_analysis = cort_samples.copy()
display(cort_analysis.head())

#############################################
display(Markdown("##### Statistics"))

steps = [("prep", "normality"), ("prep", "equal_var"), ("test", "mixed_anova"), ("posthoc", "pairwise_ttests")]
params = {"dv": "cortisol", "within": "sample", "between": "condition", "subject": "subject"}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_interaction.xlsx"))
stats.display_results(prep=False)

#############################################
display(Markdown("##### Latex Output"))
display(stats.stats_to_latex("mixed_anova"))

#### Cortisol Features

In [ ]:
display(Markdown("##### Prepare Data"))

features = ["auc_g", "auc_i", "auc_i_post", "max_inc", "slopeS1S4"]

cort_analysis = cort_features.loc[pd.IndexSlice[:, :, features]].copy()
display(cort_analysis.head())

######################################################
display(Markdown("##### Statistics"))

steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests"),
]
params = {"dv": "cortisol", "between": "condition", "groupby": "saliva_feature", "test__parametric": False}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_features.xlsx"))
stats.display_results()


######################################################
display(Markdown("##### Latex Output"))

caption = "T-tests of cortisol measures between Control and CFT condition."
index_rename_map = {
    "auc_g": "$AUC_{G}$",
    "auc_i": "$AUC_{I}$",
    "slopeS1S4": "$a_{S1S4}$",
    "max_inc": "$\Delta c_{max}$"
}

index_value_order = {
    "saliva_feature": [
        "auc_g",
        "auc_i",
        "max_inc",
        "slopeS1S4"
    ]
}

df_latex = stats.results_to_latex_table(
    "pairwise_ttests",
    index_kws={
        "index_value_order": index_value_order,
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"],
    },
    caption=caption,
    label="tab:tab_ttest_cortisol_measures",
    position="h",
    column_format="l||lSS"
)

print(stats.stats_to_latex("pairwise_ttests"))
print(df_latex)

for path in [tex_path, paper_tex_path]:
    if path is not None:
        path.joinpath("tab_ttest_saliva_features.tex").open(mode="w+").write(df_latex)


##### Plots

In [ ]:
features = {"auc_g": ["auc_g"], "auc_i": ["auc_i"], "max_inc": ["max_inc"], "slope": ["slopeS1S4"]}

box_pairs, pvalues = stats.sig_brackets(
    "test", stats_effect_type="between", plot_type="multi", x="saliva_feature", features=features, subplots=True
)

fig, axs = plt.subplots(ncols=len(features.keys()))

bp.protocols.plotting.saliva_multi_feature_boxplot(
    cort_features,
    "cortisol",
    features=features,
    hue="condition",
    hue_order=hue_order,
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    palette=cmaps.faculties,
    axs=axs,
)

for path in [plot_path, paper_img_path]:
    fig.savefig(path.joinpath("img_cortisol_features.pdf"), transparent=True)